In [1]:
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from langchain.llms import VertexAI
from vertexai.language_models import TextGenerationModel
import pandas as pd
import json
import datetime

* **1. Get API Keys:**

    * Visit the Google Cloud Credential Console: https://console.cloud.google.com/apis/credentials
    * Create a GOOGLE_API_KEY for your project.
    * Head to the Programmable Search Engine: https://programmablesearchengine.google.com/controlpanel/create
    * Generate a GOOGLE_CSE_ID for your search engine.
                

* **2. Store Keys in a json:**

    * load json file and pass keys to  fetch_parent_company function

In [2]:
#load json file 
with open('keys.json') as f:
    keys = json.load(f)

In [13]:
from modules.rag_base import RagBase
from modules.dq_engine import DqEngine

In [14]:
# Establish a connection to the BigQuery database
engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')

# Instantiate a RagBase object, associating it with the BigQuery engine
rb = RagBase(engine)
dqe = DqEngine(engine)

/tmp/ipykernel_5682/2249894983.py:2: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pybigquery.sqlalchemy_bigquery.BigQueryDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')


In [5]:
# Calling `load_data` function with specific parameters:
df = rb.load_data(schema_name='test_datasets',
                   table_name='sample_data_gdrive',
                   uids=['asin','category','title'], 
                   target_attribute='brand_parent',
                   source_attribute ='brand',
                   exclude_scored ='output.*'
                   )
df.shape

(20, 5)

In [6]:

test_df = df.iloc[[2,3],:]
test_df

,asin,category,title,brand_parent,brand
2,B010Q32SLY,Prime_Pantry,"C2O Pure Coconut Water, With Pulp, 17.5 Ounce ...","C2O Coconut Water, LLC",Not all coconut waters are created equal. C2O ...
3,B00OTH6HCA,Prime_Pantry,EAS Myoplex Original Ready-to-Drink Nutrition ...,Abbott Laboratories,EAS


In [7]:
# Create a VertexAI Language Model object

# Specify the model name: 'text-bison@001' (replace with your specific model)
llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=256,
    temperature=0,
    top_k=1,
    verbose=True,
)

/home/user/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.vertexai.VertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(


In [8]:
# Retrieve parent companies for the loaded data using the RagBase object:
response = rb.retrieve_data(data_frame=test_df,
                           llm=llm,
                           access_keys=keys,
                           prompt="what is the parent company of {x}? Respond with only the parent company name. \
                                   Do not provide any other explanations or details in your response, just the company name.")

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
/home/user/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/home/user/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I don't know the answer to this question. I will try to find it.
Action: Google Search
Action Input: c2o coconut water parent company
Observation: Oct 10, 2019 ... Under co-CEOs Ron Greene (C2O Coconut Water) and Linda Barron (Steaz) the sales structure has been realigned to be managed by former C2O VP of ...
Thought:I now know the final answer
Final Answer: Steaz

> Finished chain.


> Entering new AgentExecutor chain...
I need to find the parent company of EAS
Action: Google Search
Action Input: parent company of EAS
Observation: Mr. Steele acquired the Food, Dietary Supplement, and Cosmetic consulting divisions and Founded · Mr. · EAS is acquired by Certified Laboratories, expanding ...
Thought:I found the parent company of EAS
Final Answer: Certified Laboratories

> Finished chain.


/home/user/ItemDataQuality/itemdataqualityrepo/modules/rag_base.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['TARGET_ATTRIBUTE_model_value'] = data_frame[self.source_attribute].apply(


In [9]:
#Extracted response in dataframe
response

,asin,category,title,TARGET_ATTRIBUTE_catalog_value,TARGET_ATTRIBUTE_source_attributes,TARGET_ATTRIBUTE_model_value,TARGET_ATTRIBUTE_confidence_score
0,B010Q32SLY,Prime_Pantry,"C2O Pure Coconut Water, With Pulp, 17.5 Ounce ...","C2O Coconut Water, LLC",Not all coconut waters are created equal. C2O ...,Steaz,0.95
1,B00OTH6HCA,Prime_Pantry,EAS Myoplex Original Ready-to-Drink Nutrition ...,Abbott Laboratories,EAS,Certified Laboratories,0.95


In [10]:
# Retrieve parent companies for the loaded data using the RagBase object:
results = rb.retrieve_data(data_frame=df,
                           llm=llm,
                           access_keys=keys,
                           prompt="what is the parent company of {x}? Respond with only the parent company name. Do not provide any other explanations or details in your response, just the company name.")

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0




> Entering new AgentExecutor chain...
I need to find the parent company of V8
Action: Google Search
Action Input: "parent company of V8"
Observation: Since 1948, the brand has been owned by the Campbell Soup Company. V8 Vegetable Juice. V8 juices exhibited at a store in ...
Thought:I now know the final answer
Final Answer: Campbell Soup Company

> Finished chain.


> Entering new AgentExecutor chain...
I don't know what 47783 is
Action: Google Search
Action Input: 47783
Observation: Find 47783 and related products for scientific research at MilliporeSigma.
Thought:I found that 47783 is a product
Action: Google Search
Action Input: MilliporeSigma parent company
Observation: ... business and in combination with Merck's earlier acquired Millipore, operates as MilliporeSigma. ... 1989 – Fluka Chemie AG (Swiss company founded in the 1950s) ...
Thought:I found that MilliporeSigma is a subsidiary of Merck
Action: Google Search
Action Input: Merck parent company
Observation: The Merck Group 

In [15]:
dqe.write_dqresults(results, 
                uids=['asin','category','title'], 
                target_attribute='brand_parent', 
                function_name='retrieval', 
                error_id=30002, 
                error_code="Accuracy : Extracted supplier/vendor name from model doesn't match provided name",
                output_schema='output')

INFO:root:  **Writing results to BigQuery table: brand_parent_retrieval_2024288840**


11 out of 11 rows loaded.00<?, ?it/s]INFO:pandas_gbq.gbq:
100%|██████████| 1/1 [00:00<00:00, 1177.51it/s]
